In [1]:
import torch
import torchvision

In [10]:
class Model(torch.nn.Module):
    def __init__(self,n_features):
        super(Model,self).__init__()
        self.l1 = torch.nn.Linear(n_features,2)
        
    def forward(self,x):
        x = self.l1(x)
        return x

In [11]:
model = Model(6)

## Types of Load and Save

#### Type 1: Entire model using torch.save(model,FILE_PATH) 

In [12]:
import os
model_dir = './saved_models'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    
##------------Saving---------------------
FILE_PATH = f'{model_dir}/model_trial.pth'
torch.save(model,FILE_PATH)

In [14]:
##-------------Loading--------------------
model = torch.load(FILE_PATH)
model.eval()
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[-0.2305,  0.0329, -0.3841,  0.1831, -0.0626,  0.1721],
        [ 0.0443, -0.0983, -0.1339,  0.0677,  0.0040,  0.2238]],
       requires_grad=True)
Parameter containing:
tensor([-0.3289, -0.3627], requires_grad=True)


#### Type 2: model State Dict using torch.save(model.state_dict(),FILE_PATH) 

In [15]:
import os
model_dir = './saved_models'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    
##------------Saving---------------------
FILE_PATH = f'{model_dir}/model_trial_state_dict.pth'
torch.save(model.state_dict(),FILE_PATH)

In [16]:
## -----------Loading-------------------
## Extra but safe step of initilializing the model
model_load = Model(6)
model_load.load_state_dict(torch.load(FILE_PATH))
model_load.eval()

Model(
  (l1): Linear(in_features=6, out_features=2, bias=True)
)

In [17]:
for param in model_load.parameters():
    print(param)

Parameter containing:
tensor([[-0.2305,  0.0329, -0.3841,  0.1831, -0.0626,  0.1721],
        [ 0.0443, -0.0983, -0.1339,  0.0677,  0.0040,  0.2238]],
       requires_grad=True)
Parameter containing:
tensor([-0.3289, -0.3627], requires_grad=True)


#### Type3: Using Checkpoint dictionary

In [18]:
epochs = 10
optimizer = torch.optim.SGD(model.parameters(),lr = 0.01)
checkpoint = {
    'epoch': epochs,
    'model_state': model.state_dict(),
    'optimizer': optimizer.state_dict()
}

In [19]:
import os
model_dir = './saved_models'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    
##------------Saving---------------------
FILE_PATH = f'{model_dir}/model_trial_checkpoint.pth'
torch.save(checkpoint,FILE_PATH)

In [25]:
## -----------Loading--------------------
checkpoint_loaded = torch.load(FILE_PATH)
# print(checkpoint_loaded)
epoch = checkpoint_loaded['epoch']
# print(epoch)
##-----------Loading model-----------------
model = Model(6)
model.load_state_dict(checkpoint_loaded['model_state'])
model.eval()
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[-0.2305,  0.0329, -0.3841,  0.1831, -0.0626,  0.1721],
        [ 0.0443, -0.0983, -0.1339,  0.0677,  0.0040,  0.2238]],
       requires_grad=True)
Parameter containing:
tensor([-0.3289, -0.3627], requires_grad=True)


## Saving on GPU, Loading on CPU

In [27]:
import os
device = torch.device('cuda')
model.to(device)


model_dir = './saved_models'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    
##------------Saving---------------------
FILE_PATH = f'{model_dir}/model_trial_gpu_cpu.pth'
torch.save(model.state_dict(),FILE_PATH)

##-----------Loading---------------------
device = torch.device('cpu')
model = Model(6)
model.load_state_dict(torch.load(FILE_PATH,map_location=device))
model.eval()

for param in model.parameters():
    print(param)

Parameter containing:
tensor([[-0.2305,  0.0329, -0.3841,  0.1831, -0.0626,  0.1721],
        [ 0.0443, -0.0983, -0.1339,  0.0677,  0.0040,  0.2238]],
       requires_grad=True)
Parameter containing:
tensor([-0.3289, -0.3627], requires_grad=True)


## Saving on GPU , Loading on GPU

In [28]:
import os

device = torch.device('cuda')
model.to(device)

model_dir = './saved_models'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    
##------------Saving---------------------
FILE_PATH = f'{model_dir}/model_trial_gpu_gpu.pth'
torch.save(model.state_dict(),FILE_PATH)

model = Model(6)
model.load_state_dict(torch.load(FILE_PATH))
model.to(device)
model.eval()

Model(
  (l1): Linear(in_features=6, out_features=2, bias=True)
)

## Saving on CPU , Loading on GPU

In [29]:
model_dir = './saved_models'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    
FILE_PATH = f'{model_dir}/model_trial_cpu_gpu.pth'
torch.save(model.state_dict(),FILE_PATH)

device = torch.device('cuda')
model = Model(6)
model.load_state_dict(torch.load(FILE_PATH,map_location='cuda:0'))
model.to(device)
model.eval()

Model(
  (l1): Linear(in_features=6, out_features=2, bias=True)
)